In [15]:
import pandas as pd
import numpy as np
import time
import re

In [16]:
df=pd.read_csv('../Data/萧山房源.csv')

In [17]:
df1=df.drop_duplicates()

In [18]:
data=df1.drop(['标题','介绍','套内面积（平米）','大区域','建造时间','梯户比例'],axis=1)

In [19]:
data['挂牌时间/秒']=data['挂牌时间'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d')))

data=data.drop(data[data['上次交易时间']=='暂无数据'].index)
data['上次交易时间/秒']=data['上次交易时间'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d')))
data=data.drop(['挂牌时间','上次交易时间'],axis=1)

In [20]:
def layout_handle(layout):
    result = []
    content = re.match('^(\d)室(\d)厅(\d)厨(\d)卫', layout)
    if not content:
        return [np.nan,np.nan,np.nan,np.nan]
        print('!')
    for i in range(1, 5):
        result.append(content.group(i))
    return result


def floor_handle(floor):
    result = []
    content = re.match('^(.)楼层\s\(共(\d*)层\)', floor)
    if not content:
        return [np.nan,np.nan]
        print('!')
    for i in range(1, 3):
        result.append(content.group(i))
    return result

In [21]:
data['楼层高度']=data['所在楼层'].apply(lambda x:floor_handle(x)[0])
data['总楼层数']=data['所在楼层'].apply(lambda x:floor_handle(x)[1])
data['卧室数']=data['户型'].apply(lambda x:layout_handle(x)[0])
data['客厅数']=data['户型'].apply(lambda x:layout_handle(x)[1])
data['厨房数']=data['户型'].apply(lambda x:layout_handle(x)[2])
data['厕所数']=data['户型'].apply(lambda x:layout_handle(x)[3])

In [22]:
data=data.drop(['所在楼层','户型'],axis=1)

In [23]:
data=data.replace('暂无数据',np.nan)

In [24]:
data.isnull().any()

小区          False
价格（万元）       True
平米单价（元）     False
建筑面积（平米）    False
小区域         False
朝向          False
装修情况        False
户型结构         True
建筑类型         True
建筑结构        False
房屋用途        False
房屋年限        False
产权所属        False
配备电梯         True
挂牌时间/秒      False
上次交易时间/秒    False
楼层高度        False
总楼层数        False
卧室数         False
客厅数         False
厨房数         False
厕所数         False
dtype: bool

In [25]:
print('价格（万元）缺失值：',data['价格（万元）'].isnull().sum())
print('户型结构缺失值：',data['户型结构'].isnull().sum())
print('建筑类型缺失值：',data['建筑类型'].isnull().sum())
print('配备电梯缺失值：',data['配备电梯'].isnull().sum())

价格（万元）缺失值： 32
户型结构缺失值： 178
建筑类型缺失值： 36
配备电梯缺失值： 114


In [26]:
data['户型结构']=data['户型结构'].fillna('平层')
data['配备电梯']=data['配备电梯'].fillna('无')
data['建筑类型']=data['建筑类型'].fillna('板楼')
data=data.dropna(how='any')

In [27]:
print('价格（万元）缺失值：',data['价格（万元）'].isnull().sum())
print('户型结构缺失值：',data['户型结构'].isnull().sum())
print('建筑类型缺失值：',data['建筑类型'].isnull().sum())
print('配备电梯缺失值：',data['配备电梯'].isnull().sum())

价格（万元）缺失值： 0
户型结构缺失值： 0
建筑类型缺失值： 0
配备电梯缺失值： 0


In [28]:
data.head(3)

,小区,价格（万元）,平米单价（元）,建筑面积（平米）,小区域,朝向,装修情况,户型结构,建筑类型,建筑结构,...,产权所属,配备电梯,挂牌时间/秒,上次交易时间/秒,楼层高度,总楼层数,卧室数,客厅数,厨房数,厕所数
0,融信保利创世纪,760.0,85346,89.05,钱江世纪城,南,精装,平层,板楼,钢混结构,...,共有,有,1.610554e+09,1.539187e+09,中,32,3,2,1,2
1,德意中兴广场,225.0,17340,129.76,萧山新城区,南,精装,平层,板楼,钢混结构,...,非共有,有,1.599926e+09,1.216224e+09,中,30,3,2,1,2
2,保利霞飞郡,399.0,47620,83.79,萧山新城区,南,精装,平层,塔楼,钢混结构,...,共有,有,1.610208e+09,1.473437e+09,低,26,3,2,1,1


In [29]:
data.to_csv('../Data/萧山房源数据整理版.csv',index=False,encoding="utf-8-sig")